# Desafios Numpy - Loop Vetorizado


A ideia de loop vetorizado é substituir for explícitos por operações de NumPy, aproveitando que essas bibliotecas processam arrays em baixo nível (C/Fortran), deixando o código mais rápido e legível.


👉 Sugestão: tente sempre com NumPy (np.where, broadcasting, slicing, máscaras booleanas, np.add.outer, np.cumsum, etc).

In [1]:
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view

In [2]:
array_random_int = np.random.randint(1, 100, 10)
array_random_int

array([65, 63, 29, 17, 90, 64, 39, 60,  5, 53], dtype=int32)

## 1. Soma acumulada de uma série
Dado um array `np.arange(1, 11)`, calcule a soma acumulada ([1, 3, 6, 10, ...]) sem usar laço for.

In [3]:
array = np.arange(1,11)
# Usando a função
soma_acumulada_f = array.cumsum()
# Reproduzindo com matriz
soma_acumulada_s = np.multiply(np.tri(len(array)), array).sum(axis=1)
soma_acumulada_f, soma_acumulada_s

(array([ 1,  3,  6, 10, 15, 21, 28, 36, 45, 55]),
 array([ 1.,  3.,  6., 10., 15., 21., 28., 36., 45., 55.]))

⚡ **Resposta otimizada**  
✅ A versão com `cumsum` já é a forma mais direta e eficiente. Sua versão com matriz é criativa, mas menos legível e mais custosa.

In [4]:
array = np.arange(1, 11)
# cumsum faz a soma acumulada em C, muito rápido e legível
soma_acumulada = array.cumsum()
soma_acumulada

array([ 1,  3,  6, 10, 15, 21, 28, 36, 45, 55])

## 2. Diferença entre elementos consecutivos
Crie um array de inteiros aleatórios e calcule a diferença entre cada elemento e o anterior.

In [5]:
diferencas_s = array_random_int[1:] - array_random_int[:-1]
# Usando função
diferencas_f = np.diff(array_random_int)
diferencas_s, diferencas_f

(array([ -2, -34, -12,  73, -26, -25,  21, -55,  48], dtype=int32),
 array([ -2, -34, -12,  73, -26, -25,  21, -55,  48], dtype=int32))

⚡ **Resposta otimizada**  
✅ Sua versão com slicing já está correta e vetorizada. `np.diff` é equivalente, mas mais legível.

In [6]:
# slicing vetorizado
diferencas = array_random_int[1:] - array_random_int[:-1]
# versão mais legível com função pronta
diferencas_f = np.diff(array_random_int)
diferencas_f

array([ -2, -34, -12,  73, -26, -25,  21, -55,  48], dtype=int32)

## 3. Normalização de um vetor
Dado um array, normalize todos os valores para estarem entre 0 e 1.

In [7]:
normalizacao_s = (array_random_int - min(array_random_int)) / (max(array_random_int) - min(array_random_int))
np.round(normalizacao_s, 2)

array([0.71, 0.68, 0.28, 0.14, 1.  , 0.69, 0.4 , 0.65, 0.  , 0.56])

⚡ **Resposta otimizada**  
⚠️ Use sempre `.min()` e `.max()` do NumPy em vez de `min`/`max` do Python. São mais rápidos e mantêm o tipo do array.
Além disso, adicione um guardinha contra divisão por zero.

In [8]:
# A normalização min–max leva todos os valores para o intervalo [0, 1]:
# (x - min) / (max - min)
# Use SEMPRE os métodos do NumPy (.min() / .max()) em vez de min/max do Python.
# - Eles operam em C sobre o buffer do array (sem converter para lista).
# - São mais rápidos e mantêm o tipo/shape corretos.

x_min = array_random_int.min()   # escalar NumPy (mais rápido)
x_max = array_random_int.max()

den = x_max - x_min              # amplitude (ptp). Poderíamos usar array_random_int.ptp()

# Guardinha: se todos os valores forem iguais, den == 0.
# Evita divisão por zero, retornando zeros (ou poderíamos retornar NaN, a depender do caso).
normalizacao = np.where(den == 0, 0.0, (array_random_int - x_min) / den)

# Observação:
# Mesmo que array_random_int seja int, a divisão gera float (dtype float64 por padrão).
x_min, x_max, den, np.round(normalizacao, 2)

(np.int32(5),
 np.int32(90),
 np.int32(85),
 array([0.71, 0.68, 0.28, 0.14, 1.  , 0.69, 0.4 , 0.65, 0.  , 0.56]))

## 4. Contagem condicional
Dado um array de inteiros, conte quantos são pares e quantos são ímpares sem usar loops.

In [9]:
print(f"Pare: {np.sum(array_random_int % 2 == 0)}\nÍmpares: {np.sum(array_random_int % 2 == 1)}")

Pare: 3
Ímpares: 7


⚡ **Resposta otimizada**  
✅ Sua solução já está ótima e vetorizada. Apenas separe em variáveis em vez de print direto para reutilização.

In [10]:
pares = np.sum(array_random_int % 2 == 0)
impares = np.sum(array_random_int % 2 == 1)
pares, impares

(np.int64(3), np.int64(7))

## 5. Máscara booleana
Dado um array `np.arange(20)`, filtre apenas os números múltiplos de 3 sem loop.

In [11]:
array = np.arange(20)
multiplos_de_3 = array[array % 3 == 0]
multiplos_de_3

array([ 0,  3,  6,  9, 12, 15, 18])

⚡ **Resposta otimizada**  
✅ Sua resposta já está correta, vetorizada e eficiente.

In [12]:
array = np.arange(20)
multiplos_de_3 = array[array % 3 == 0]
multiplos_de_3

array([ 0,  3,  6,  9, 12, 15, 18])

## 6. Produto cartesiano vetorizado
Dadas duas listas [1, 2, 3] e [10, 20, 30], gere todas as combinações possíveis de soma de forma vetorizada.

In [13]:
produto_cartesiano = [1, 2, 3] + np.array([[10, 20, 30]]*len([1, 2, 3])).T
produto_cartesiano

array([[11, 12, 13],
       [21, 22, 23],
       [31, 32, 33]])

⚡ **Resposta otimizada**  
❌ A solução original não gera um verdadeiro produto cartesiano. Use `np.add.outer` ou broadcasting explícito.

In [14]:
a = np.array([1, 2, 3])
b = np.array([10, 20, 30])

# np.add.outer faz: resultado[i, j] = a[i] + b[j]
# Shape: (len(a), len(b)) → (3, 3)
produto_cartesiano = np.add.outer(a, b)

# Equivalente por broadcasting explícito:
# a[:, None] tem shape (3, 1)
# b[None, :] tem shape (1, 3)
# A soma "espalha" para (3, 3) gerando todas as combinações.
produto_cartesiano_alt = a[:, None] + b[None, :]

# Se você quiser um vetor 1D com todas as somas (em vez de matriz 2D),
# pode "achatar" com ravel():
todas_as_somas = produto_cartesiano.ravel()

# Por que a versão original estava incorreta?
# Ela somava uma lista [1,2,3] a um array modelado, produzindo
# broadcasting inesperado e não o produto cartesiano de somas.
produto_cartesiano, produto_cartesiano_alt, todas_as_somas

(array([[11, 21, 31],
        [12, 22, 32],
        [13, 23, 33]]),
 array([[11, 21, 31],
        [12, 22, 32],
        [13, 23, 33]]),
 array([11, 21, 31, 12, 22, 32, 13, 23, 33]))

## 7. Distância euclidiana entre pontos
Dada uma matriz n x 2 representando coordenadas (x, y), calcule todas as distâncias euclidianas entre os pontos (sem loop duplo).

In [15]:
matrix = np.random.randint(0,5,16).reshape(8,2)
# Com slicing
distancias_euclidianas_s = np.sqrt(np.sum(((matrix[1:] - matrix[:-1])**2), axis=1))
# Com função própria
distancias_euclidianas_f = np.linalg.norm(np.diff(matrix, axis=0), axis=1)
matrix, np.round(distancias_euclidianas_s, 4), np.round(distancias_euclidianas_f, 4)

(array([[4, 3],
        [1, 2],
        [3, 1],
        [2, 2],
        [0, 4],
        [3, 4],
        [4, 2],
        [4, 4]], dtype=int32),
 array([3.1623, 2.2361, 1.4142, 2.8284, 3.    , 2.2361, 2.    ]),
 array([3.1623, 2.2361, 1.4142, 2.8284, 3.    , 2.2361, 2.    ]))

⚡ **Resposta otimizada**  
❌ Seu código calcula apenas distâncias entre consecutivos. Para todas as distâncias (matriz n x n), use broadcasting.

In [16]:
# Matriz n x 2 com coordenadas (x, y)
matrix = np.random.randint(0, 5, 16).reshape(8, 2)  # n=8 pontos

# Queremos a matriz de distâncias (n x n), onde D[i, j] = ||p_i - p_j||.
# 1) Expandimos dimensões para preparar o broadcasting:
#    - matrix[:, None, :] → shape (n, 1, 2)
#    - matrix[None, :, :] → shape (1, n, 2)
# 2) A subtração produz um tensor (n, n, 2) com todos os pares de diferenças.
# 3) Elevamos ao quadrado, somamos no eixo das coordenadas (axis=2) e aplicamos sqrt.

diffs = matrix[:, None, :] - matrix[None, :, :]  # (n, n, 2)
sq_dists = (diffs ** 2).sum(axis=2)              # (n, n) — distâncias ao quadrado
distancias = np.sqrt(sq_dists)                   # (n, n) — distâncias euclidianas

# Notas:
# - A diagonal é zero (distância do ponto para ele mesmo).
# - Complexidade de memória/tempo é O(n^2). Para n muito grande,
#   considere estratégias por blocos ou funções especializadas (ex.: scipy.spatial.distance.cdist).
# - Se você só quer a parte superior da matriz (sem duplicar pares), use np.triu_indices.
sq_dists, np.round(distancias, 4)

(array([[ 0, 10,  5, 16, 20, 17, 20,  5],
        [10,  0,  5, 18, 10,  1,  2,  5],
        [ 5,  5,  0,  5,  5, 10,  9,  0],
        [16, 18,  5,  0,  4, 25, 20,  5],
        [20, 10,  5,  4,  0, 13,  8,  5],
        [17,  1, 10, 25, 13,  0,  1, 10],
        [20,  2,  9, 20,  8,  1,  0,  9],
        [ 5,  5,  0,  5,  5, 10,  9,  0]]),
 array([[0.    , 3.1623, 2.2361, 4.    , 4.4721, 4.1231, 4.4721, 2.2361],
        [3.1623, 0.    , 2.2361, 4.2426, 3.1623, 1.    , 1.4142, 2.2361],
        [2.2361, 2.2361, 0.    , 2.2361, 2.2361, 3.1623, 3.    , 0.    ],
        [4.    , 4.2426, 2.2361, 0.    , 2.    , 5.    , 4.4721, 2.2361],
        [4.4721, 3.1623, 2.2361, 2.    , 0.    , 3.6056, 2.8284, 2.2361],
        [4.1231, 1.    , 3.1623, 5.    , 3.6056, 0.    , 1.    , 3.1623],
        [4.4721, 1.4142, 3.    , 4.4721, 2.8284, 1.    , 0.    , 3.    ],
        [2.2361, 2.2361, 0.    , 2.2361, 2.2361, 3.1623, 3.    , 0.    ]]))

## 8. Frequência de valores
Dado um array aleatório de inteiros entre 0 e 9, conte quantas vezes cada número aparece (sem for).

In [17]:
array = np.random.randint(0,10, 1000000)
unicos = np.unique(array)
frequencia_s = [np.count_nonzero(array == n) for n in unicos]
array, unicos, frequencia_s

(array([0, 4, 7, ..., 1, 7, 7], shape=(1000000,), dtype=int32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 [100493, 100018, 99998, 99657, 100633, 100027, 99682, 100165, 99445, 99882])

⚡ **Resposta otimizada**  
❌ Evite `for`. Use `np.unique(..., return_counts=True)` ou, quando os valores forem pequenos, `np.bincount`.

In [18]:
# Array de inteiros em [0, 9]
array = np.random.randint(0, 10, 1_000_000)

# Maneira direta, vetorizada e idiomática:
# unique_vals → os valores únicos encontrados
# counts      → quantas vezes cada um aparece (na mesma ordem)
unique_vals, counts = np.unique(array, return_counts=True)

# Quando você conhece previamente que os valores são pequenos e não-negativos,
# np.bincount costuma ser AINDA mais rápido:
counts_fast = np.bincount(array, minlength=10)  # retorna vetor de contagens de 0..9
# Se quiser os "valores" explícitos junto, você mesmo cria:
vals_fast = np.arange(counts_fast.size)         # [0, 1, ..., 9]

# Observação de performance/memória:
# Arrays com 100 milhões de elementos (~100_000_000) podem consumir muita RAM (≈800 MB para float64).
# Aqui usei 1_000_000 para manter o exemplo enxuto e rápido.
array, unique_vals, counts, counts_fast, vals_fast

(array([4, 6, 9, ..., 4, 6, 5], shape=(1000000,), dtype=int32),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32),
 array([100345, 100034, 100041,  99385, 100197,  99765,  99688, 100005,
        100207, 100333]),
 array([100345, 100034, 100041,  99385, 100197,  99765,  99688, 100005,
        100207, 100333]),
 array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]))

## 9. Substituição condicional
Dado um array de notas [5, 7, 9, 3, 10], transforme em APROVADO se ≥ 7, senão REPROVADO, sem loop.

In [19]:
notas = np.array([5, 7, 9, 3, 10])
situacao = np.array(["Não calculado"]*5)
mask = notas >= 7
situacao[mask] = 'Aprovado'
situacao[~mask] = 'Reprovado'
situacao

array(['Reprovado', 'Aprovado', 'Aprovado', 'Reprovado', 'Aprovado'],
      dtype='<U13')

⚡ **Resposta otimizada**  
⚠️ Use `np.where` diretamente, mais legível e vetorizado.

In [20]:
notas = np.array([5, 7, 9, 3, 10])

# np.where(cond, valor_se_True, valor_se_False) aplica elemento a elemento.
# Aqui a condição é (notas >= 7).
# O resultado é um array de strings (dtype <U... / object), pois os ramos são textos.
situacao = np.where(notas >= 7, "Aprovado", "Reprovado")

# Vantagens:
# - Não precisa criar um array de saída e sobrescrever com máscaras separadas.
# - Código mais legível e "declarativo".
# - 100% vetorizado: sem laços Python.
situacao

array(['Reprovado', 'Aprovado', 'Aprovado', 'Reprovado', 'Aprovado'],
      dtype='<U9')

## 10. Média móvel vetorizada
Dado um array de 100 números, calcule a média móvel de janela 5 de forma vetorizada.

In [21]:
janela = np.ones(5) / 5 # média móvel de tamanho 5
array = np.random.randint(0,5, 100)

media_movel_s = sliding_window_view(array, window_shape=5).mean(axis=1)
media_movel_convolve = np.convolve(array, janela, mode='valid')
media_movel_correlate = np.correlate(array, janela, mode='valid')
janela, array, media_movel_s, media_movel_convolve, media_movel_correlate

(array([0.2, 0.2, 0.2, 0.2, 0.2]),
 array([3, 1, 3, 4, 3, 4, 4, 3, 2, 3, 4, 2, 4, 4, 4, 2, 1, 1, 2, 0, 0, 4,
        4, 2, 3, 3, 4, 1, 2, 0, 3, 0, 2, 3, 4, 0, 3, 0, 4, 2, 2, 4, 2, 1,
        0, 2, 2, 1, 3, 3, 0, 0, 3, 3, 0, 2, 2, 0, 4, 4, 1, 0, 0, 4, 2, 4,
        1, 0, 0, 1, 3, 1, 2, 0, 0, 1, 4, 0, 4, 0, 3, 1, 3, 2, 4, 1, 2, 2,
        3, 3, 0, 0, 4, 4, 2, 1, 0, 2, 4, 1], dtype=int32),
 array([2.8, 3. , 3.6, 3.6, 3.2, 3.2, 3.2, 2.8, 3. , 3.4, 3.6, 3.2, 3. ,
        2.4, 2. , 1.2, 0.8, 1.4, 2. , 2. , 2.6, 3.2, 3.2, 2.6, 2.6, 2. ,
        2. , 1.2, 1.4, 1.6, 2.4, 1.8, 2.4, 2. , 2.2, 1.8, 2.2, 2.4, 2.8,
        2.2, 1.8, 1.8, 1.4, 1.2, 1.6, 2.2, 1.8, 1.4, 1.8, 1.8, 1.2, 1.6,
        2. , 1.4, 1.6, 2.4, 2.2, 1.8, 1.8, 1.8, 1.4, 2. , 2.2, 2.2, 1.4,
        1.2, 1. , 1. , 1.4, 1.4, 1.2, 0.8, 1.4, 1. , 1.8, 1.8, 2.2, 1.6,
        2.2, 1.8, 2.6, 2.2, 2.4, 2.2, 2.4, 2.2, 2. , 1.6, 2. , 2.2, 2. ,
        2.2, 2.2, 1.8, 1.8, 1.6]),
 array([2.8, 3. , 3.6, 3.6, 3.2, 3.2, 3.2, 2.8, 3. , 3.4, 3.6, 3

⚡ **Resposta otimizada**  
✅ Sua solução já está correta e vetorizada. `sliding_window_view`, `convolve` e `correlate` são todas válidas.

In [22]:
janela = np.ones(5) / 5
array = np.random.randint(0,5, 100)
media_movel_sliding = sliding_window_view(array, window_shape=5).mean(axis=1)
media_movel_convolve = np.convolve(array, janela, mode='valid')
media_movel_correlate = np.correlate(array, janela, mode='valid')
janela, array, media_movel_sliding, media_movel_convolve, media_movel_correlate

(array([0.2, 0.2, 0.2, 0.2, 0.2]),
 array([3, 4, 2, 1, 4, 1, 3, 2, 1, 3, 2, 3, 4, 3, 1, 0, 3, 1, 3, 3, 0, 0,
        0, 2, 3, 3, 0, 2, 0, 3, 4, 1, 3, 2, 0, 3, 2, 2, 0, 4, 0, 3, 1, 0,
        3, 1, 1, 0, 2, 4, 2, 2, 4, 4, 0, 2, 2, 3, 2, 0, 1, 1, 4, 2, 3, 2,
        3, 2, 3, 1, 0, 2, 3, 4, 4, 1, 0, 1, 3, 3, 2, 1, 4, 3, 1, 2, 2, 4,
        4, 3, 1, 0, 2, 0, 2, 2, 3, 4, 2, 2], dtype=int32),
 array([2.8, 2.4, 2.2, 2.2, 2.2, 2. , 2.2, 2.2, 2.6, 3. , 2.6, 2.2, 2.2,
        1.6, 1.6, 2. , 2. , 1.4, 1.2, 1. , 1. , 1.6, 1.6, 2. , 1.6, 1.6,
        1.8, 2. , 2.2, 2.6, 2. , 1.8, 2. , 1.8, 1.4, 2.2, 1.6, 1.8, 1.6,
        1.6, 1.4, 1.6, 1.2, 1. , 1.4, 1.6, 1.8, 2. , 2.8, 3.2, 2.4, 2.4,
        2.4, 2.2, 1.8, 1.8, 1.6, 1.4, 1.6, 1.6, 2.2, 2.4, 2.8, 2.4, 2.6,
        2.2, 1.8, 1.6, 1.8, 2. , 2.6, 2.8, 2.4, 2. , 1.8, 1.6, 1.8, 2. ,
        2.6, 2.6, 2.2, 2.2, 2.4, 2.4, 2.6, 3. , 2.8, 2.4, 2. , 1.2, 1. ,
        1.2, 1.8, 2.2, 2.6, 2.6]),
 array([2.8, 2.4, 2.2, 2.2, 2.2, 2. , 2.2, 2.2, 2.6, 3. , 2.6, 2